In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rl-assignment/samplesubmission.csv


In [2]:
import gym

env = gym.make("Taxi-v3").env

env.reset() # reset environment to a new, random state
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [3]:
# action space
print("Action Space {}".format(env.action_space))

print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


In [4]:
# state encoding

state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()


State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [5]:
# rewards table
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [6]:
# Defining q table

import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [7]:
# hyperparameters
alpha = 0.1
gamma = 0.6
epsiolon = 0.1

In [8]:
import random

max_epochs = 1000000 # stops after x epochs or max_iterations
max_iterations = -1 # 1000000 # -1 to not stop based on iterations

iteration = 0
for epoch in range(max_epochs):
    done = False
    new_state = env.reset()
    
    while not done and (iteration < max_iterations or max_iterations == -1):
        iteration += 1
        old_state = new_state
        if random.uniform(0,1) < epsiolon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[old_state])
        
        new_state, reward, done, info = env.step(action)
        env.s = new_state # update state of the environment (not sure if )
        
        q_table[old_state, action] = q_table[old_state][action] + alpha * (reward + gamma * np.max(q_table[new_state])  - q_table[old_state][action])
    
    if iteration >= max_iterations and max_iterations != -1:
        break
print("Epoch: {} Iterations: {}\r".format(epoch, iteration))

Epoch: 999999 Iterations: 14810058


In [9]:
q_table

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ -2.41837066,  -2.3639511 ,  -2.41837066,  -2.3639511 ,
         -2.27325184, -11.3639511 ],
       [ -1.870144  ,  -1.45024   ,  -1.870144  ,  -1.45024   ,
         -0.7504    , -10.45024   ],
       ...,
       [ -0.75955193,   0.416     ,  -0.75945955,  -1.44283176,
         -9.1295566 ,  -9.26328213],
       [ -2.26879216,  -2.1220864 ,  -2.24661946,  -2.1220864 ,
        -10.7816137 , -10.43290413],
       [  5.6       ,   2.36      ,   5.6       ,  11.        ,
         -3.4       ,  -3.4       ]])

In [10]:
# example game

new_state = env.reset()

done = False
max_iterations = 100
iteration = 0
while not done and (iteration <= max_iterations or max_iterations == -1):
    old_state = new_state
    action = np.argmax(q_table[old_state])
    new_state, reward, done, info = env.step(action)
    env.s = new_state
    env.render()
    iteration += 1


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : |_: |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : :_: |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | :_:G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y

In [11]:
#generate output file
df = pd.DataFrame(q_table.ravel())

df.index += 1
df.to_csv("q_table.csv", index_label="Id", header=["Value"])
df

# load data again to test correct file format
pd.read_csv("q_table.csv")

,Id,Value
0,1,0.00
1,2,0.00
2,3,0.00
3,4,0.00
4,5,0.00
...,...,...
2995,2996,2.36
2996,2997,5.60
2997,2998,11.00
2998,2999,-3.40
